In [2]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

df = pd.read_csv("crime.csv") 


In [3]:
df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str
df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()

In [4]:
df_other = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LAT','GEO_LON']]
df_other = df_other.loc[df_other.OFFENSE_CATEGORY_ID == 'all-other-crimes'].copy()
df_other_am =  df_other.loc[df_other.TIME.str[-2:] == "AM" ].copy()
df_other_am['TIME'] = [x[:-3] for x in df_other_am['TIME']]
df_other_am['TIME'] =  pd.to_datetime(df_other_am['TIME'], format='%H:%M:%S')

df_other = df_other[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LAT','GEO_LON']]
df_other = df_other.loc[df_other.OFFENSE_CATEGORY_ID == 'all-other-crimes'].copy()
df_other_pm =  df_other.loc[df_other.TIME.str[-2:] == "PM" ].copy()
df_other_pm['TIME'] = [x[:-3] for x in df_other_pm['TIME']]
df_other_pm['TIME'] =  pd.to_datetime(df_other_pm['TIME'], format='%H:%M:%S')
df_other_pm['TIME'] = df_other_pm['TIME'] + timedelta(hours=12)


In [5]:
final_other = pd.concat([df_other_pm, df_other_am], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [6]:
final_other.loc[final_other['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_other.loc[(final_other['TIME'].dt.time > datetime_object1.time())&(final_other['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_other.loc[(final_other['TIME'].dt.time > datetime_object2.time())&(final_other['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_other.loc[(final_other['TIME'].dt.time > datetime_object3.time())&(final_other['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_other.loc[(final_other['TIME'].dt.time > datetime_object4.time())&(final_other['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_other.loc[(final_other['TIME'].dt.time > datetime_object5.time())&(final_other['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_other['TIME'] = [d.time() for d in final_other['TIME']]
final_other

OFFENSE_CATEGORY_ID      TIME day_of_week  NEIGHBORHOOD_ID    GEO_LAT  \
0        all-other-crimes  23:59:00   Wednesday        montbello  39.773188   
1        all-other-crimes  16:00:00      Monday     lincoln-park  39.740117   
2        all-other-crimes  04:00:00     Tuesday   elyria-swansea  39.783741   
3        all-other-crimes  20:00:00      Sunday        stapleton  39.789301   
4        all-other-crimes  20:00:00     Tuesday              cbd  39.740226   
...                   ...       ...         ...              ...        ...   
94812    all-other-crimes  04:00:00      Friday       globeville  39.764054   
94813    all-other-crimes  16:00:00    Saturday         westwood  39.711266   
94814    all-other-crimes  16:00:00    Saturday  university-park  39.677890   
94815    all-other-crimes  04:00:00    Saturday         westwood  39.705519   
94816    all-other-crimes  04:00:00    Saturday            baker  39.720944   

          GEO_LON  
0     -104.809881  
1     -104.998855  
2     -104.949562  
3     -104.883245  
4     -104.994132  
...           ...  
94812 -104.994722  
94813 -105.043262  
94814 -104.941236  
94815 -105.040864  
94816 -104.987453  

[94817 rows x 6 columns]

In [13]:
other_csv = final_other[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
other_plot = final_other[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LAT','GEO_LON']]

other_csv = other_csv.sample(40000)

other_csv.to_csv('All-Other-Crimes.csv')
other_plot.to_csv('All-Other-Crimes_Plot.csv')

!python apriori.py -f 'All-Other-Crimes.csv' -s 0.002

item: ('civic-center', '23:59:00', 'Thursday') , 0.002
item: ('civic-center', '16:00:00', 'Thursday') , 0.002
item: ('cbd', '20:00:00', 'Saturday') , 0.002
item: ('20:00:00', 'Wednesday', 'east-colfax') , 0.002
item: ('westwood', 'Sunday', '23:59:00') , 0.002
item: ('montbello', '23:59:00', 'Friday') , 0.002
item: ('20:00:00', 'five-points', 'Friday') , 0.002
item: ('westwood', 'Friday', '23:59:00') , 0.002
item: ('23:59:00', 'five-points', 'Friday') , 0.002
item: ('montbello', '16:00:00', 'Thursday') , 0.002
item: ('montbello', 'Friday', '16:00:00') , 0.002
item: ('westwood', 'Saturday', '23:59:00') , 0.002
item: ('westwood', '23:59:00', 'Thursday') , 0.002
item: ('20:00:00', 'five-points', 'Saturday') , 0.002
item: ('16:00:00', 'Monday', 'east-colfax') , 0.002
item: ('montbello', '20:00:00', 'Tuesday') , 0.002
item: ('montbello', 'Wednesday', '16:00:00') , 0.002
item: ('23:59:00', 'civic-center', 'Wednesday') , 0.002
item: ('16:00:00', 'civic-center', 'Wednesday') , 0.002
item: ('cbd